In [328]:
import numpy as np
import copy
import bayes_logistic as bl
from sklearn import datasets

In [329]:
# Digit we want to classify
DIGIT = 1

In [330]:
# Prep the MNIST image classification data. The data is divided into training, 
# cross validation (cv) and test sets

TRAINING_PERCENTAGE = 0.6
CV_PERCENTAGE = 0.2
TEST_PERCENTAGE  = 0.2

digits = datasets.load_digits()
n_samples, n_features = digits.data.shape

# Add bias column to the feature matrix
X = np.ones((n_samples, n_features + 1))
X[:, 1:] = digits.data

# Change the vector to binary data
y = digits.target
for idx in range(0, y.shape[0]):
    y[idx] = 1.0 if y[idx] == DIGIT else 0.0

training_cnt = int(n_samples*TRAINING_PERCENTAGE)
training_X = X[:training_cnt,:]
training_y = y[:training_cnt]

cv_cnt = int(n_samples*CV_PERCENTAGE)
cv_X = X[training_cnt:(training_cnt + cv_cnt),:]
cv_y = y[training_cnt:(training_cnt + cv_cnt)]

test_cnt = int(n_samples*TEST_PERCENTAGE)
test_X = X[(training_cnt + cv_cnt):,:]
test_y = y[(training_cnt + cv_cnt):]


In [331]:
# Train the model 
GD_BATCH_SIZE = training_cnt
ITERATION_CNT = 1
w = training_X.shape[1]
w_prior = np.zeros(w)
H_prior = np.diag(np.ones(w))*0.001

for i in range(0, ITERATION_CNT):
    for idx in range(0, training_cnt, GD_BATCH_SIZE):
        batch_size = GD_BATCH_SIZE if (idx + GD_BATCH_SIZE) < training_cnt else training_cnt - idx
        w_posterior, H_posterior = bl.fit_bayes_logistic(training_y[idx:batch_size], 
                                                     training_X[idx:batch_size,:], 
                                                     w_prior, H_prior, solver = 'BFGS')
        w_prior = copy.copy(w_posterior)
        H_prior = copy.copy(H_posterior)
    
    # Perform Cross validation
    y_cnt = 0
    cv_probs = bl.bayes_logistic_prob(cv_X, w_prior, H_prior)
    prediction_cnt = 0
    for idx in range(0, cv_cnt):
        if cv_probs[idx] > 0.9 and cv_y[idx] == 1:
            prediction_cnt += 1
        y_cnt += 1
        
    prediction_accuracy = (100.0*prediction_cnt)/y_cnt
    print "Prediction Accuracy for cross validation set %.02f" % prediction_accuracy

Prediction Accuracy for cross validation set 8.64


In [332]:
# Perform Test
y_cnt = 0
test_probs = bl.bayes_logistic_prob(test_X, w_prior, H_prior)
prediction_cnt = 0
for idx in range(0, test_cnt):
    print idx
    if cv_probs[idx] > 0.9 and cv_y[idx] == 1:
        prediction_cnt += 1
    y_cnt += 1
        
prediction_accuracy = (100.0*prediction_cnt)/y_cnt
print "Prediction Accuracy for test set %.02f" % prediction_accuracy

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27